# Grouping

## 구간을 나눠서 그룹화 해보기

In [1]:
import pandas as pd


# 기업별 재무재표 데이터
df = pd.read_csv("../data/naver_finance/2016_12.csv")
# "price" 는 해당 년 12월 마지막날의 종가
# "price2" 는 다음 년도의 12월 마지막날의 종가
df.head(3)

,ticker,매출액(억원),영업이익률(%),순이익률(%),당기순이익(억원),ROE(%),ROA(%),ROIC(%),EPS(원),BPS(원),SPS(원),PER(배),PBR(배),PSR(배),price,price2
0,AK홀딩스,29218.3100,7.313,4.563,1333.2234,12.193,5.091,15.5147,5436.4126,48112.402,220556.1600,10.30091,1.16394,0.25390,56000.0,68500.0
1,BGF,860.7727,9.315,214.481,1846.1921,21.624,10.433,8.0153,3703.5774,18648.623,1737.2633,22.75690,4.51947,48.51420,42140.0,15250.0
2,BNK금융지주,49126.7600,14.499,10.546,5181.1440,7.919,0.564,NaN,1568.2368,20810.660,15358.9930,5.53488,0.41709,0.56514,8680.0,9420.0


In [2]:
# 수익률 구하기(2016.12 ~ 2017.12)
df["rfn"] = df["price2"] / df["price"] - 1
df.head(3)

,ticker,매출액(억원),영업이익률(%),순이익률(%),당기순이익(억원),ROE(%),ROA(%),ROIC(%),EPS(원),BPS(원),SPS(원),PER(배),PBR(배),PSR(배),price,price2,rfn
0,AK홀딩스,29218.3100,7.313,4.563,1333.2234,12.193,5.091,15.5147,5436.4126,48112.402,220556.1600,10.30091,1.16394,0.25390,56000.0,68500.0,0.223214
1,BGF,860.7727,9.315,214.481,1846.1921,21.624,10.433,8.0153,3703.5774,18648.623,1737.2633,22.75690,4.51947,48.51420,42140.0,15250.0,-0.638111
2,BNK금융지주,49126.7600,14.499,10.546,5181.1440,7.919,0.564,NaN,1568.2368,20810.660,15358.9930,5.53488,0.41709,0.56514,8680.0,9420.0,0.085253


In [3]:
# PER 에 따라서 그룹을 나눠서 그룹 번호 넘어주기

# PER 이 10을 넘는 행만 True 인 Series
bound1 = df["PER(배)"] >= 10

# PER 이 5 이상 10 미만인 행만 True 인 Series
bound2 = (df["PER(배)"] >= 5) & (df["PER(배)"] < 10)

# PER 이 0 이상 5 미만인 True 인 Series
bound3 = (df["PER(배)"] >= 0) & (df["PER(배)"] < 5)

# PER 이 0 미만인 행만 True 인 Series
bound4 = df["PER(배)"] < 0

bound1.sum(), bound2.sum(), bound3.sum(), bound4.sum()

(np.int64(378), np.int64(148), np.int64(23), np.int64(120))

In [4]:
# loc 를 사용해서 set operation 하기('=' 를 사용해서 해당 위치에 값을 넣음)
# loc 을 사용해서, "PER_Score" 열에 값을 덮어씌운다.
# 현재는 "PER_Score" 이 없으니까 새로운 열을 만들어지고 default 로 모두 NaN 이 들어감,
# 그 뒤, set operation 진행!
# 해당되는 행에만 값을 수정함
df.loc[bound1, "PER_Score"] = 1
df.loc[bound2, "PER_Score"] = 2
df.loc[bound3, "PER_Score"] = 3
df.loc[bound4, "PER_Score"] = -1

In [5]:
df["PER_Score"].head(3)

0    1.0
1    1.0
2    2.0
Name: PER_Score, dtype: float64

In [6]:
df["PER_Score"].nunique()

4

In [7]:
# PER_Score 의 value_counts() 가 float 으로 나온다!
# int 값으로 넣어줬는데, float? ==> NaN 인 값이 있다는 말!!
df["PER_Score"].value_counts()

PER_Score
 1.0    378
 2.0    148
-1.0    120
 3.0     23
Name: count, dtype: int64

In [8]:
df["PER_Score"].isna().sum()

np.int64(12)

In [9]:
# bound1 ~ bound4 가 하나도 안 겹치도록 했음에도,
# PER_Score 에 NaN 이 있다.
# => 연산을 한 피연산자 중에 NaN 이 있는 것!
# => PER(배) 중 에 NaN 인 값이 있다.
df["PER(배)"].isna().sum()

np.int64(12)

In [10]:
# PER_Score 중 NaN 값으로 0 으로 메꾸기
df[df["PER_Score"].isna()] = 0
df["PER_Score"].value_counts()

PER_Score
 1.0    378
 2.0    148
-1.0    120
 3.0     23
 0.0     12
Name: count, dtype: int64

In [11]:
# 위에서 했던 것을 1줄로 끝내기!
# 연산을 하면서 False 는 0 이 되므로 NaN 이 자동으로 0 으도 된다!
df.loc[:, "PER_Score2"] = (bound1 * 1) + (bound2 * 2) + (bound3 * 3) + (bound4 * -1)

In [12]:
df["PER_Score2"].value_counts()

PER_Score2
 1    378
 2    148
-1    120
 3     23
 0     12
Name: count, dtype: int64

In [13]:
# 둘 컬럼이 다른 이유는 dtype 이 다르기 때문에!
# equals 는 dtype 도 같고 값도 같아야 True 를 반환
df["PER_Score"].equals(df["PER_Score2"])

False

## pd.cut 을 사용해서 구간을 나눠보기

In [14]:
import numpy as np
import pandas as pd


# 기업별 재무재표 데이터
df = pd.read_csv("../data/naver_finance/2016_12.csv")
# "price" 는 해당 년 12월 마지막날의 종가
# "price2" 는 다음 년도의 12월 마지막날의 종가
df.head(3)

,ticker,매출액(억원),영업이익률(%),순이익률(%),당기순이익(억원),ROE(%),ROA(%),ROIC(%),EPS(원),BPS(원),SPS(원),PER(배),PBR(배),PSR(배),price,price2
0,AK홀딩스,29218.3100,7.313,4.563,1333.2234,12.193,5.091,15.5147,5436.4126,48112.402,220556.1600,10.30091,1.16394,0.25390,56000.0,68500.0
1,BGF,860.7727,9.315,214.481,1846.1921,21.624,10.433,8.0153,3703.5774,18648.623,1737.2633,22.75690,4.51947,48.51420,42140.0,15250.0
2,BNK금융지주,49126.7600,14.499,10.546,5181.1440,7.919,0.564,NaN,1568.2368,20810.660,15358.9930,5.53488,0.41709,0.56514,8680.0,9420.0


In [15]:
per_cuts = pd.cut(
    df["PER(배)"],
    [-np.inf, 0, 5, 10, np.inf],
)
# dtype 이 catetory!
print(type(per_cuts))
per_cuts.head()

<class 'pandas.core.series.Series'>


0    (10.0, inf]
1    (10.0, inf]
2    (5.0, 10.0]
3    (10.0, inf]
4    (10.0, inf]
Name: PER(배), dtype: category
Categories (4, interval[float64, right]): [(-inf, 0.0] < (0.0, 5.0] < (5.0, 10.0] < (10.0, inf]]

In [16]:
per_cuts[0]  # pandas 의 Interval 을 값으로 갖고 있다.

Interval(10.0, inf, closed='right')

In [17]:
per_cuts.value_counts()

PER(배)
(10.0, inf]    378
(5.0, 10.0]    148
(-inf, 0.0]    120
(0.0, 5.0]      23
Name: count, dtype: int64

In [18]:
# cut 으로 그룹화를 할 때도,
# NaN 은 크기 비교가 불가해서 그대로 NaN 이 된 것!
per_cuts.isna().sum()

np.int64(12)

In [19]:
bins = [-np.inf, 10, 20, np.inf]
labels = ["저평가", "보통주", "고평가주"]
per_cuts2 = pd.cut(df["PER(배)"], bins, labels=labels)
per_cuts2.head()

0     보통주
1    고평가주
2     저평가
3     보통주
4    고평가주
Name: PER(배), dtype: category
Categories (3, object): ['저평가' < '보통주' < '고평가주']

In [20]:
df.loc[:, "PER_Score2"] = per_cuts2
df.head()

,ticker,매출액(억원),영업이익률(%),순이익률(%),당기순이익(억원),ROE(%),ROA(%),ROIC(%),EPS(원),BPS(원),SPS(원),PER(배),PBR(배),PSR(배),price,price2,PER_Score2
0,AK홀딩스,29218.3100,7.313,4.563,1333.22340,12.193,5.091,15.5147,5436.4126,48112.402,220556.1600,10.30091,1.16394,0.25390,56000.0,68500.0,보통주
1,BGF,860.7727,9.315,214.481,1846.19210,21.624,10.433,8.0153,3703.5774,18648.623,1737.2633,22.75690,4.51947,48.51420,42140.0,15250.0,고평가주
2,BNK금융지주,49126.7600,14.499,10.546,5181.14400,7.919,0.564,NaN,1568.2368,20810.660,15358.9930,5.53488,0.41709,0.56514,8680.0,9420.0,저평가
3,BYC,2118.5762,7.625,8.281,175.43266,4.543,2.463,10.7475,20872.3120,471887.000,252211.4500,19.02041,0.84130,1.57408,397000.0,306000.0,보통주
4,CJ,239541.9700,5.230,2.379,5698.23400,6.080,2.253,5.1579,6257.1523,114276.080,672045.9000,28.18128,1.54306,0.26238,176334.0,171148.0,고평가주


## pd.qcut 을 사용해서 구간 나눠보기

In [21]:
import pandas as pd


# 기업별 재무재표 데이터
df = pd.read_csv("../data/naver_finance/2016_12.csv")
# "price" 는 해당 년 12월 마지막날의 종가
# "price2" 는 다음 년도의 12월 마지막날의 종가
df.head(3)

,ticker,매출액(억원),영업이익률(%),순이익률(%),당기순이익(억원),ROE(%),ROA(%),ROIC(%),EPS(원),BPS(원),SPS(원),PER(배),PBR(배),PSR(배),price,price2
0,AK홀딩스,29218.3100,7.313,4.563,1333.2234,12.193,5.091,15.5147,5436.4126,48112.402,220556.1600,10.30091,1.16394,0.25390,56000.0,68500.0
1,BGF,860.7727,9.315,214.481,1846.1921,21.624,10.433,8.0153,3703.5774,18648.623,1737.2633,22.75690,4.51947,48.51420,42140.0,15250.0
2,BNK금융지주,49126.7600,14.499,10.546,5181.1440,7.919,0.564,NaN,1568.2368,20810.660,15358.9930,5.53488,0.41709,0.56514,8680.0,9420.0


In [22]:
# quantile cut!
pd.qcut(df["PER(배)"], 3)

0          (7.666, 16.495]
1       (16.495, 7056.129]
2      (-27857.497, 7.666]
3       (16.495, 7056.129]
4       (16.495, 7056.129]
              ...         
676     (16.495, 7056.129]
677    (-27857.497, 7.666]
678     (16.495, 7056.129]
679    (-27857.497, 7.666]
680    (-27857.497, 7.666]
Name: PER(배), Length: 681, dtype: category
Categories (3, interval[float64, right]): [(-27857.497, 7.666] < (7.666, 16.495] < (16.495, 7056.129]]

In [23]:
pd.qcut(df["PER(배)"], 3, labels=[1, 2, 3])

0      2
1      3
2      1
3      3
4      3
      ..
676    3
677    1
678    3
679    1
680    1
Name: PER(배), Length: 681, dtype: category
Categories (3, int64): [1 < 2 < 3]

In [24]:
df.loc[:, "PER_Score2"] = pd.qcut(df["PER(배)"], 10, labels=range(1, 11))
df.head()

,ticker,매출액(억원),영업이익률(%),순이익률(%),당기순이익(억원),ROE(%),ROA(%),ROIC(%),EPS(원),BPS(원),SPS(원),PER(배),PBR(배),PSR(배),price,price2,PER_Score2
0,AK홀딩스,29218.3100,7.313,4.563,1333.22340,12.193,5.091,15.5147,5436.4126,48112.402,220556.1600,10.30091,1.16394,0.25390,56000.0,68500.0,5
1,BGF,860.7727,9.315,214.481,1846.19210,21.624,10.433,8.0153,3703.5774,18648.623,1737.2633,22.75690,4.51947,48.51420,42140.0,15250.0,8
2,BNK금융지주,49126.7600,14.499,10.546,5181.14400,7.919,0.564,NaN,1568.2368,20810.660,15358.9930,5.53488,0.41709,0.56514,8680.0,9420.0,3
3,BYC,2118.5762,7.625,8.281,175.43266,4.543,2.463,10.7475,20872.3120,471887.000,252211.4500,19.02041,0.84130,1.57408,397000.0,306000.0,8
4,CJ,239541.9700,5.230,2.379,5698.23400,6.080,2.253,5.1579,6257.1523,114276.080,672045.9000,28.18128,1.54306,0.26238,176334.0,171148.0,9


In [25]:
# 거의 비슷한 갯수로 구간이 나눠져 있다.
df["PER_Score2"].value_counts()

PER_Score2
1     67
2     67
3     67
4     67
5     67
7     67
8     67
9     67
10    67
6     66
Name: count, dtype: int64

In [26]:
df["PER_Score2"].dtype

CategoricalDtype(categories=range(1, 11), ordered=True, categories_dtype=int64)

In [27]:
# qcut 으로 PER_Score2 를 만들었기 때문에,
# NaN 도 pd.CategoricalDtype 으로 채워야한다.
df.dropna(subset=["PER_Score2"], inplace=True)
df["PER_Score2"].hasnans

False

## groupby & aggregation

In [28]:
import pandas as pd


df = pd.read_csv("../data/naver_finance/2016_12.csv")
df.head(3)

,ticker,매출액(억원),영업이익률(%),순이익률(%),당기순이익(억원),ROE(%),ROA(%),ROIC(%),EPS(원),BPS(원),SPS(원),PER(배),PBR(배),PSR(배),price,price2
0,AK홀딩스,29218.3100,7.313,4.563,1333.2234,12.193,5.091,15.5147,5436.4126,48112.402,220556.1600,10.30091,1.16394,0.25390,56000.0,68500.0
1,BGF,860.7727,9.315,214.481,1846.1921,21.624,10.433,8.0153,3703.5774,18648.623,1737.2633,22.75690,4.51947,48.51420,42140.0,15250.0
2,BNK금융지주,49126.7600,14.499,10.546,5181.1440,7.919,0.564,NaN,1568.2368,20810.660,15358.9930,5.53488,0.41709,0.56514,8680.0,9420.0


In [29]:
# 여기서는 편의를 위해 결측치를 그냥 버림
# 원본은 유지한채 조작할 것을 새로 만들기!
g_df = df.dropna().copy()

In [30]:
g_df["rtn"] = g_df["price2"] / g_df["price"] - 1
g_df.loc[:, "PER_Score"] = pd.qcut(g_df["PER(배)"], 10, labels=range(1, 11))
g_df.loc[:, "PBR_Score"] = pd.qcut(g_df["PBR(배)"], 10, labels=range(1, 11))
# 종목명은 인덱스로 지정!
g_df.set_index("ticker", inplace=True)
g_df.head()

,매출액(억원),영업이익률(%),순이익률(%),당기순이익(억원),ROE(%),ROA(%),ROIC(%),EPS(원),BPS(원),SPS(원),PER(배),PBR(배),PSR(배),price,price2,rtn,PER_Score,PBR_Score
ticker,,,,,,,,,,,,,,,,,,
AK홀딩스,29218.3100,7.313,4.563,1333.22340,12.193,5.091,15.5147,5436.4126,48112.402,220556.1600,10.30091,1.16394,0.25390,56000.0,68500.0,0.223214,5,7
BGF,860.7727,9.315,214.481,1846.19210,21.624,10.433,8.0153,3703.5774,18648.623,1737.2633,22.75690,4.51947,48.51420,42140.0,15250.0,-0.638111,8,10
BYC,2118.5762,7.625,8.281,175.43266,4.543,2.463,10.7475,20872.3120,471887.000,252211.4500,19.02041,0.84130,1.57408,397000.0,306000.0,-0.229219,8,5
CJ,239541.9700,5.230,2.379,5698.23400,6.080,2.253,5.1579,6257.1523,114276.080,672045.9000,28.18128,1.54306,0.26238,176334.0,171148.0,-0.029410,9,8
CJ CGV,14322.4540,4.911,0.393,56.21660,3.105,0.284,2.9189,583.0513,17654.154,67682.2600,120.74409,3.98773,1.04015,70400.0,74200.0,0.053977,10,10


In [31]:
g_df.dtypes.value_counts()

float64     16
category     2
Name: count, dtype: int64

In [32]:
# groupby() 는 실제로 grouping 까지는 하지 않고,
# grouping 이 가능한지 validation 만 진행(즉, preparation 정도)
# groupby() 를 한 뒤, (1)
# 특정 column 을 기준으로, (2)
# aggregation function 을 호출(3)해줘야
# 연산으로서 의미가 있는 것!

# `observed=False` (현재 기본값):
#     - 모든 카테고리 수준을 결과에 포함, 실제 데이터에 존재하지 않는 카테고리도 포함
# `observed=True` (향후 기본값):
#     - 데이터에서 관찰된 카테고리만 결과에 포함

g_df_obj = g_df.groupby(["PBR_Score", "PER_Score"], observed=True)
g_df_obj

In [33]:
g_df_obj.ngroups

96

In [34]:
g_df["PBR_Score"].nunique()

10

In [35]:
g_df["PER_Score"].nunique()

10

In [36]:
# 왜 ngroups와
# (g_df['PBR_score'].nunique() x g_df['PER_score'].nunique()) 가
# 차이가 날까?
# 특정 PBR_Score category 에 속하는 것 중 PER_Score 에 속하지 않을 수 있어서!
# 예를 들어, 다음과 같은 상황이 있을 수 있다
# - `PBR_Score=1, PER_Score=1`인 회사가 있음
# - `PBR_Score=1, PER_Score=2`인 회사가 있음
# - 하지만 `PBR_Score=1, PER_Score=3`인 회사는 데이터셋에 존재하지 않음

g_df_obj.ngroups == (g_df["PBR_Score"].nunique() * g_df["PER_Score"].nunique())

False

In [37]:
# 전체 그룹 확인
# g_df_obj.size() 는 Multi-level index를 가진 Series
print(type(g_df_obj.size()))
g_df_obj.size()

<class 'pandas.core.series.Series'>


PBR_Score  PER_Score
1          1             5
           2            11
           3            11
           4            11
           5             7
                        ..
10         6             1
           7             2
           8            15
           9            11
           10           14
Length: 96, dtype: int64

In [38]:
print(type(g_df_obj.groups))
g_df_obj.groups.keys()

<class 'pandas.io.formats.printing.PrettyDict'>


dict_keys([(1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 9), (1, 10), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (3, 10), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (4, 10), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (5, 6), (5, 7), (5, 8), (5, 9), (5, 10), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (6, 6), (6, 7), (6, 8), (6, 9), (6, 10), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (7, 6), (7, 7), (7, 8), (7, 9), (7, 10), (8, 1), (8, 2), (8, 3), (8, 4), (8, 5), (8, 6), (8, 7), (8, 8), (8, 9), (8, 10), (9, 1), (9, 2), (9, 4), (9, 5), (9, 6), (9, 7), (9, 8), (9, 9), (9, 10), (10, 1), (10, 2), (10, 4), (10, 5), (10, 6), (10, 7), (10, 8), (10, 9), (10, 10)])

In [39]:
# iterating 할 수 있음!!
for name, group in g_df_obj:
    print(name)  # index
    print(group.head(3))  # 그 그룹에 속하는 data
    break

(np.int64(1), np.int64(1))
           매출액(억원)  영업이익률(%)  순이익률(%)   당기순이익(억원)  ROE(%)  ROA(%)  ROIC(%)  \
ticker                                                                       
E1      39959.0080     0.277   -0.949 -379.396850  -2.350  -1.076  -1.6326   
S&T중공업   4680.0776    -0.309   -5.221 -244.357940  -3.671  -2.697  -0.7910   
디아이동일    8224.1450     2.100    0.132   10.885012  -0.753   0.111   0.0509   

           EPS(원)      BPS(원)     SPS(원)    PER(배)   PBR(배)   PSR(배)    price  \
ticker                                                                          
E1     -3869.0110  191789.690  582492.80 -16.20569  0.32692  0.10764  62700.0   
S&T중공업  -734.8515   21043.785   14074.28 -12.36877  0.43192  0.64579   9089.0   
디아이동일  -1523.4067  270773.030  333554.28 -37.36512  0.21022  0.17065  56922.0   

         price2       rtn PER_Score PBR_Score  
ticker                                         
E1      56900.0 -0.092504         1         1  
S&T중공업   7310.0 -0.195731       

In [40]:
# 참고 :groupby()에 대해 head()를 적용하면, 기존이 head()가 작동하는 방식
# 즉, 최상위 2개를 가지고 오는게 아니라
# 각 그룹별 최상위 2개를 무작위로 섞어서 하나로 합친 DataFrame을 리턴함
g_df.groupby("PBR_Score", observed=True).head(2)

,매출액(억원),영업이익률(%),순이익률(%),당기순이익(억원),ROE(%),ROA(%),ROIC(%),EPS(원),BPS(원),SPS(원),PER(배),PBR(배),PSR(배),price,price2,rtn,PER_Score,PBR_Score
ticker,,,,,,,,,,,,,,,,,,
AK홀딩스,29218.3100,7.313,4.563,1333.22340,12.193,5.091,15.5147,5436.41260,48112.4020,220556.1600,10.30091,1.16394,0.25390,56000.0,68500.0,0.223214,5,7
BGF,860.7727,9.315,214.481,1846.19210,21.624,10.433,8.0153,3703.57740,18648.6230,1737.2633,22.75690,4.51947,48.51420,42140.0,15250.0,-0.638111,8,10
BYC,2118.5762,7.625,8.281,175.43266,4.543,2.463,10.7475,20872.31200,471887.0000,252211.4500,19.02041,0.84130,1.57408,397000.0,306000.0,-0.229219,8,5
CJ,239541.9700,5.230,2.379,5698.23400,6.080,2.253,5.1579,6257.15230,114276.0800,672045.9000,28.18128,1.54306,0.26238,176334.0,171148.0,-0.029410,9,8
CJ CGV,14322.4540,4.911,0.393,56.21660,3.105,0.284,2.9189,583.05130,17654.1540,67682.2600,120.74409,3.98773,1.04015,70400.0,74200.0,0.053977,10,10
CJ대한통운,60819.4570,3.756,1.122,682.10130,2.390,1.361,4.9474,2445.65770,133105.0800,266607.6600,73.19095,1.34480,0.67140,179000.0,140000.0,-0.217877,10,8
CJ씨푸드,1737.3776,3.372,3.093,53.72999,10.205,5.037,6.9419,148.70976,1530.1201,4808.5810,23.33404,2.26780,0.72163,3470.0,2875.0,-0.171470,8,9
CS홀딩스,1231.4496,16.105,13.918,171.38786,4.409,5.921,12.7189,7994.88600,186451.8400,106666.8500,10.09395,0.43282,0.75656,80700.0,80200.0,-0.006196,5,1
DB,2048.0996,5.324,-10.113,-207.12779,-13.052,-8.434,66.5738,-113.72754,881.1775,1124.5490,-6.45402,0.83298,0.65271,734.0,658.0,-0.103542,2,5


## aggregation

- min
- max
- mean
- median
- sum
- var
- size
- nunique
- idxmax

In [41]:
import numpy as np
import pandas as pd


df = pd.read_csv("../data/naver_finance/2016_12.csv")

# 여기서는 편의를 위해 결측치를 그냥 버림
# 원본은 유지한채 조작할 것을 새로 만들기!
g_df = df.dropna().copy()

In [42]:
g_df["rtn"] = g_df["price2"] / g_df["price"] - 1
g_df.loc[:, "PER_Score"] = pd.qcut(g_df["PER(배)"], 10, labels=range(1, 11))
g_df.loc[:, "PBR_Score"] = pd.qcut(g_df["PBR(배)"], 10, labels=range(1, 11))

# 종목명은 인덱스로 지정!
g_df.set_index("ticker", inplace=True)
g_df.head(3)

,매출액(억원),영업이익률(%),순이익률(%),당기순이익(억원),ROE(%),ROA(%),ROIC(%),EPS(원),BPS(원),SPS(원),PER(배),PBR(배),PSR(배),price,price2,rtn,PER_Score,PBR_Score
ticker,,,,,,,,,,,,,,,,,,
AK홀딩스,29218.3100,7.313,4.563,1333.22340,12.193,5.091,15.5147,5436.4126,48112.402,220556.1600,10.30091,1.16394,0.25390,56000.0,68500.0,0.223214,5,7
BGF,860.7727,9.315,214.481,1846.19210,21.624,10.433,8.0153,3703.5774,18648.623,1737.2633,22.75690,4.51947,48.51420,42140.0,15250.0,-0.638111,8,10
BYC,2118.5762,7.625,8.281,175.43266,4.543,2.463,10.7475,20872.3120,471887.000,252211.4500,19.02041,0.84130,1.57408,397000.0,306000.0,-0.229219,8,5


In [43]:
g_df_obj = g_df.groupby(["PBR_Score", "PER_Score"], observed=True)

In [44]:
# agg 를 쓸 때는,
# 기준으로 할 열 이름과, 적용할 함수(aggregation 함수만!)를 지정하면된다!
# dict, list 등등 다 할 수 있음, 아래와 같이
# g_df.groupby("PBR_Score", observed=True).agg({
#     "rtn": ["mean", "std"],
#     "PER(배)": ["min"]
# })
pbr_rtn_df = g_df.groupby("PBR_Score", observed=True).agg({"rtn": "mean"})
pbr_rtn_df

,rtn
PBR_Score,
1,-0.001363
2,0.020453
3,-0.020788
4,0.160985
5,-0.011614
6,-0.043329
7,0.150012
8,0.057876
9,0.139403


In [45]:
# 다음과 같이도 할 수 있음
# g_df.groupby("PER_Score", observed=True)["rtn"].agg("mean")
# g_df.groupby("PER_Score", observed=True)["rtn"].agg(np.mean)
# g_df.groupby("PER_Score", observed=True)["rtn"].mean()
per_rtn_df = g_df.groupby("PER_Score", observed=True).agg({"rtn": "mean"})
per_rtn_df

,rtn
PER_Score,
1,-0.061915
2,-0.083212
3,-0.037584
4,0.056213
5,0.000077
6,0.095373
7,0.150638
8,0.144230
9,0.047995


In [46]:
ex_df = g_df.groupby(["PBR_Score", "PER_Score"], observed=True).agg(
    {"rtn": ["mean", "std"], "ROE(%)": ["mean", "size", "nunique", "idxmax"]}
)
ex_df.head()

rtn                ROE(%)                       
                         mean       std        mean size nunique    idxmax
PBR_Score PER_Score                                                       
1         1         -0.099839  0.071890   -1.401800    5       5  한국수출포장공업
          2         -0.093158  0.266421  154.966727   11      11      삼부토건
          3          0.117311  0.359099    6.952455   11      11    한국전력공사
          4          0.105904  0.294823    5.104182   11      11      한국공항
          5         -0.039217  0.119516    3.941429    7       7    성창기업지주

주의! NaN 은 groupby 시 자동으로 filter out 되기 때문에, 미리 전처리 해 두는 것이 좋음!

In [47]:
df = pd.DataFrame(
    {
        "a": ["소형주", np.nan, "대형주", "대형주"],
        "b": [np.nan, 2, 3, np.nan],
    }
)
df

,a,b
0,소형주,NaN
1,NaN,2.0
2,대형주,3.0
3,대형주,NaN


In [48]:
df.groupby(["a"])["b"].mean()

a
대형주    3.0
소형주    NaN
Name: b, dtype: float64

In [49]:
# as_index 는 groupby 의 기준이 된 열을 index 로 할 것인지 아닌지를 결정하는 인자
# default 는 True => index 로 만듬!
# 기준이 2개 이상이면, Multi-index 를 만드는 것
g_df.groupby(["PER_Score"], observed=True).agg({"rtn": ["mean", "std"]}).head(2)

rtn          
               mean       std
PER_Score                    
1         -0.061915  0.327539
2         -0.083212  0.780563

In [50]:
# 기준 열을 index 로 만들기 싫으면, as_index 를 False 로!
g_df.groupby(["PER_Score"], observed=True, as_index=False).agg({"rtn": ["mean", "std"]}).head(2)

PER_Score       rtn          
                 mean       std
0         1 -0.061915  0.327539
1         2 -0.083212  0.780563

Multi-index 는 조작하기 힘드니까 level 을 줄이자!

In [51]:
ex_df.head()  # 너무 복잡하니까 한 줄로 만들겠다!

rtn                ROE(%)                       
                         mean       std        mean size nunique    idxmax
PBR_Score PER_Score                                                       
1         1         -0.099839  0.071890   -1.401800    5       5  한국수출포장공업
          2         -0.093158  0.266421  154.966727   11      11      삼부토건
          3          0.117311  0.359099    6.952455   11      11    한국전력공사
          4          0.105904  0.294823    5.104182   11      11      한국공항
          5         -0.039217  0.119516    3.941429    7       7    성창기업지주

In [52]:
level0 = ex_df.columns.get_level_values(0)
level1 = ex_df.columns.get_level_values(1)
level0, level1

(Index(['rtn', 'rtn', 'ROE(%)', 'ROE(%)', 'ROE(%)', 'ROE(%)'], dtype='object'),
 Index(['mean', 'std', 'mean', 'size', 'nunique', 'idxmax'], dtype='object'))

In [53]:
level0 + "_" + level1

Index(['rtn_mean', 'rtn_std', 'ROE(%)_mean', 'ROE(%)_size', 'ROE(%)_nunique',
       'ROE(%)_idxmax'],
      dtype='object')

In [54]:
# Multi-index 인 column 을 한 줄로!
ex_df.columns = level0 + "_" + level1
ex_df.head()

rtn_mean   rtn_std  ROE(%)_mean  ROE(%)_size  \
PBR_Score PER_Score                                                 
1         1         -0.099839  0.071890    -1.401800            5   
          2         -0.093158  0.266421   154.966727           11   
          3          0.117311  0.359099     6.952455           11   
          4          0.105904  0.294823     5.104182           11   
          5         -0.039217  0.119516     3.941429            7   

                     ROE(%)_nunique ROE(%)_idxmax  
PBR_Score PER_Score                                
1         1                       5      한국수출포장공업  
          2                      11          삼부토건  
          3                      11        한국전력공사  
          4                      11          한국공항  
          5                       7        성창기업지주

In [55]:
# Multi-index 인 index 도 태초의 모습으로
# 이렇게 간단하게 되어있어야, 다른 연산하기가 좋음.
ex_df.reset_index(inplace=True)
ex_df.head()

,PBR_Score,PER_Score,rtn_mean,rtn_std,ROE(%)_mean,ROE(%)_size,ROE(%)_nunique,ROE(%)_idxmax
0,1,1,-0.099839,0.071890,-1.401800,5,5,한국수출포장공업
1,1,2,-0.093158,0.266421,154.966727,11,11,삼부토건
2,1,3,0.117311,0.359099,6.952455,11,11,한국전력공사
3,1,4,0.105904,0.294823,5.104182,11,11,한국공항
4,1,5,-0.039217,0.119516,3.941429,7,7,성창기업지주


### Example

In [56]:
import pandas as pd


# 매우 큰 데이터로,
# 2000년 7월 ~ 2018년 6월까지
# 월말 별로 종목명, PBR, 베타, 수익률, 시가총액 을 모아둔 데이터
df = pd.read_csv("../data/Small_and_Big.csv", index_col=[0])
df.head()

,date,종목명,PBR(IFRS-연결),"베타 (M,5Yr)",수익률(%),시가총액 (보통)(평균)(원)
0,2000-07-31,BYC,0.21,0.47940,-0.58,2.778600e+10
1,2000-07-31,CJ,0.51,1.16611,-9.00,1.160889e+12
2,2000-07-31,CJ ENM,6.56,NaN,17.40,4.004670e+11
3,2000-07-31,CJ대한통운,0.17,1.31374,-7.96,1.949620e+11
4,2000-07-31,CJ씨푸드,NaN,0.22678,32.00,1.987000e+09


In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 235929 entries, 0 to 235928
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   date              235929 non-null  object 
 1   종목명               235929 non-null  object 
 2   PBR(IFRS-연결)      233991 non-null  float64
 3   베타 (M,5Yr)        214201 non-null  float64
 4   수익률(%)            235929 non-null  float64
 5   시가총액 (보통)(평균)(원)  235929 non-null  float64
dtypes: float64(4), object(2)
memory usage: 12.6+ MB


월별로 시가총액의 median 을 구해서, median 보다 크면 Big 작으면 Small 로

In [58]:
median_df = df.groupby(["date"]).agg({"시가총액 (보통)(평균)(원)": "median"})
median_df.head()

,시가총액 (보통)(평균)(원)
date,
2000-07-31,3.494700e+10
2000-08-31,3.368400e+10
2000-09-30,3.368400e+10
2000-10-31,3.052300e+10
2000-11-30,3.079800e+10


In [59]:
median_df.columns = ["시가총액_median"]
median_df.head()

,시가총액_median
date,
2000-07-31,3.494700e+10
2000-08-31,3.368400e+10
2000-09-30,3.368400e+10
2000-10-31,3.052300e+10
2000-11-30,3.079800e+10


In [60]:
g_df = df.copy()
g_df.set_index("date", inplace=True)
g_df.head()

,종목명,PBR(IFRS-연결),"베타 (M,5Yr)",수익률(%),시가총액 (보통)(평균)(원)
date,,,,,
2000-07-31,BYC,0.21,0.47940,-0.58,2.778600e+10
2000-07-31,CJ,0.51,1.16611,-9.00,1.160889e+12
2000-07-31,CJ ENM,6.56,NaN,17.40,4.004670e+11
2000-07-31,CJ대한통운,0.17,1.31374,-7.96,1.949620e+11
2000-07-31,CJ씨푸드,NaN,0.22678,32.00,1.987000e+09


In [61]:
g_df["시가총액_median"] = median_df
g_df

,종목명,PBR(IFRS-연결),"베타 (M,5Yr)",수익률(%),시가총액 (보통)(평균)(원),시가총액_median
date,,,,,,
2000-07-31,BYC,0.21,0.47940,-0.58,2.778600e+10,3.494700e+10
2000-07-31,CJ,0.51,1.16611,-9.00,1.160889e+12,3.494700e+10
2000-07-31,CJ ENM,6.56,NaN,17.40,4.004670e+11,3.494700e+10
2000-07-31,CJ대한통운,0.17,1.31374,-7.96,1.949620e+11,3.494700e+10
2000-07-31,CJ씨푸드,NaN,0.22678,32.00,1.987000e+09,3.494700e+10
...,...,...,...,...,...,...
2018-06-30,흥국에프엔비,0.95,1.36392,-1.69,6.492600e+10,1.646500e+11
2018-06-30,흥국화재,0.65,0.72064,8.89,3.831750e+11,1.646500e+11
2018-06-30,흥아해운,0.92,1.24047,-5.75,1.344730e+11,1.646500e+11


In [62]:
big_index = g_df["시가총액 (보통)(평균)(원)"] <= g_df["시가총액_median"]
big_index.head()

date
2000-07-31     True
2000-07-31    False
2000-07-31    False
2000-07-31    False
2000-07-31     True
dtype: bool

In [63]:
small_index = g_df["시가총액_median"] < g_df["시가총액 (보통)(평균)(원)"]
small_index.head()

date
2000-07-31    False
2000-07-31     True
2000-07-31     True
2000-07-31     True
2000-07-31    False
dtype: bool

In [64]:
g_df.loc[:, "Big_or_Small"] = (big_index * 1) + (small_index * -1)
g_df.head()

,종목명,PBR(IFRS-연결),"베타 (M,5Yr)",수익률(%),시가총액 (보통)(평균)(원),시가총액_median,Big_or_Small
date,,,,,,,
2000-07-31,BYC,0.21,0.47940,-0.58,2.778600e+10,3.494700e+10,1
2000-07-31,CJ,0.51,1.16611,-9.00,1.160889e+12,3.494700e+10,-1
2000-07-31,CJ ENM,6.56,NaN,17.40,4.004670e+11,3.494700e+10,-1
2000-07-31,CJ대한통운,0.17,1.31374,-7.96,1.949620e+11,3.494700e+10,-1
2000-07-31,CJ씨푸드,NaN,0.22678,32.00,1.987000e+09,3.494700e+10,1


In [68]:
g_df["Big_or_Small"].value_counts()

Big_or_Small
 1    118020
-1    117909
Name: count, dtype: int64